In [1]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

(6, 1)

In [6]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-4bd6rzvj/unsloth_7293d2addcc849bbb3b72a0f26540cb2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-4bd6rzvj/unsloth_7293d2addcc849bbb3b72a0f26540cb2
  Resolved https://github.com/unslothai/unsloth.git to commit 85f1fa096afde5efe2fb8521d8ceec8d13a00715
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... one
  Created wheel for flash-attn: filename=flash_attn-2.7.2.post1-cp311-cp311-linux_x86_64.whl size=190179680 sha256=0bd51e5504afaf3648750c23aeef78e7d04e36b21a6bb4e1efc3d62dbb3a92f0
  Stored in directory: /root/.cache/pip/wheels/6a/8b/7d/0ac2b18cb28f4104a1852da090dcf9ea8239ce45fc82bcc4d1
Successfully built flash-attn


In [7]:
import torch
from unsloth import FastLanguageModel
import huggingface_hub
import os
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from peft import LoraConfig

In [8]:
max_seq_length = 4096
dtype = None
load_in_4bit = True

In [10]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=4096,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.47.1.
   \\   /|    GPU: NVIDIA GeForce GTX 1060 6GB. Max memory: 6.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 6.1. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [6]:
EOS_TOKEN = tokenizer.eos_token
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}
### Response:
{}"""

In [8]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"] 
    outputs = examples["output"] 
    texts = [] 
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

dataset = load_dataset("prismdata/KDI-DATASET-2014", split="train")
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

README.md:   0%|          | 0.00/426 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/165k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1331 [00:00<?, ? examples/s]

Map:   0%|          | 0/1331 [00:00<?, ? examples/s]

In [10]:
tokenizer.padding_side = "right" 
Lora_config = LoraConfig(
r=16,
lora_alpha=32,
lora_dropout=0.05,
bias="none"
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,  
    train_dataset=dataset,  
    eval_dataset=dataset,
    dataset_text_field="text", 
    max_seq_length=max_seq_length, 
    dataset_num_proc=1,  
    packing=False, 
    peft_config =Lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,  
        gradient_accumulation_steps=4, 
        warmup_steps=5, 
        num_train_epochs=3,
        max_steps=100,
        do_eval=True,
        evaluation_strategy="steps",
        logging_steps=1,
        learning_rate=2e-4, 
        fp16=not torch.cuda.is_bf16_supported(), 
        bf16=torch.cuda.is_bf16_supported(), 
        optim="adamw_8bit", 
        weight_decay=0.01, 
        lr_scheduler_type="cosine", 
        seed=123, 
        output_dir="outputs", 
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/1331 [00:00<?, ? examples/s]

Map:   0%|          | 0/1331 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [11]:
from peft import LoraConfig, get_peft_model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,331 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 100
 "-____-"     Number of trainable parameters = 6,815,744


Step,Training Loss,Validation Loss
1,3.207500,No log
2,3.106000,No log
3,3.045400,No log
4,2.869500,No log
5,3.223200,No log
6,2.719500,No log
7,2.811500,No log
8,2.483700,No log
9,2.170800,No log
10,2.239900,No log


In [21]:
model.save_pretrained("Llama-3-Open-Ko-8B-prismdata")

In [19]:
#옵션 1) vLLM용

In [38]:
base_model = "beomi/Llama-3-Open-Ko-8B"  # 병합을 수행할 베이스 모델
save_method = ("merged_16bit")

In [18]:
model.save_pretrained_merged(
    base_model,
    tokenizer,
    save_method=save_method,
)

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 16.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 30.9 out of 52.96 RAM for saving.


100%|██████████| 32/32 [00:04<00:00,  7.01it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


In [22]:
#옵션 2) HuggingFace 에 업로드

In [23]:
# Hub 에 업로드
huggingface_repo = ''
huggingface_token = ''
model.push_to_hub_merged(
    huggingface_repo,
    tokenizer,
    save_method=save_method,
    token=huggingface_token,
)

Unsloth: You are pushing to hub, but you passed your HF username = prismdata.
We shall truncate prismdata/KDI-Llama-3-Open-Ko-8B-Instruct to KDI-Llama-3-Open-Ko-8B-Instruct


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 29.36 out of 52.96 RAM for saving.


100%|██████████| 32/32 [00:04<00:00,  7.11it/s]


Unsloth: Saving tokenizer...

  0%|          | 0/1 [00:00<?, ?it/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/prismdata/KDI-Llama-3-Open-Ko-8B-Instruct


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import shutil
import os

# 원본 경로
source_path = '/content/KDI-Llama-3-Open-Ko-8B-Instruct'

# 목적지 경로
destination_path = '/content/drive/MyDrive/Colab Notebooks/KDI-Llama-3-Open-Ko-8B-Instruct'

# 목적지 경로가 없으면 생성
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# '/content' 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path):
    file_path = os.path.join(source_path, filename)
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        shutil.copy(file_path, destination_path)

In [28]:
# 원본 경로
source_path = '/content/Llama-3-Open-Ko-8B-prismdata'

# 목적지 경로
destination_path = '/content/drive/MyDrive/Colab Notebooks/Llama-3-Open-Ko-8B-prismdata'

# 목적지 경로가 없으면 생성
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# '/content' 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path):
    file_path = os.path.join(source_path, filename)
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        shutil.copy(file_path, destination_path)

In [29]:
#Inference
model.config.torch_dtype = torch.bfloat16 #매우 중요..

In [70]:
#이 루틴이 가장 간단함.
FastLanguageModel.for_inference(model) 
inputs = tokenizer(
[
    alpaca_prompt.format(
         "Centrelink를 중심으로 한 호주의 공공서비스에 대해 알려줘",
        "",
    )
], return_tensors = "pt").to("cuda")

print("torch data type", model.config.torch_dtype)

outputs = model.generate(**inputs, max_new_tokens = 4096, use_cache = True)
decoded_output = tokenizer.batch_decode(outputs)
print('----------------')
print(decoded_output)

torch data type torch.bfloat16
----------------
['<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCentrelink를 중심으로 한 호주의 공공서비스에 대해 알려줘\n### Response:\nCentrelink를 중심으로 한 호주의 공공서비스는 호주의 공공서비스 전달체계를 구성하고 있습니다.<|end_of_text|>']
